In [3]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
import numpy as np

# Reading log data from file
with open('/content/output.log', 'r') as file:
    log_data = file.read()

# Parsing log entries
pattern = re.compile(r'(\d+\.\d+\.\d+\.\d+) \[([^\]]+)\] \[([^\]]+)\] (\d+) (\d+) \[\[([^\]]+)\]\] (\d+)')
matches = pattern.findall(log_data.strip())

# Creating a DataFrame
columns = ['IP', 'Timestamp', 'Request', 'Status', 'Bytes', 'UserAgent', 'Duration']
log_df = pd.DataFrame(matches, columns=columns)

# Convert numerical columns to appropriate data types
log_df['Status'] = log_df['Status'].astype(int)
log_df['Bytes'] = log_df['Bytes'].astype(int)
log_df['Duration'] = log_df['Duration'].astype(int)

# Parsing the timestamp
log_df['Timestamp'] = pd.to_datetime(log_df['Timestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')

# Extracting HTTP method and URL from the Request field
log_df[['HTTPMethod', 'URL']] = log_df['Request'].str.extract(r'(\w+) (.+)')

# Drop the original Request column
log_df.drop(columns=['Request'], inplace=True)

# Extracting user agent categories (e.g., browser, bot)
log_df['UserAgentCategory'] = log_df['UserAgent'].apply(lambda x: 'Bot' if 'bot' in x.lower() else 'Browser')

# Selecting features for the autoencoder
features = ['Status', 'Bytes', 'Duration']
X = log_df[features].values

# Standardizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Autoencoder model
input_dim = X_scaled.shape[1]
encoding_dim = 2  # Number of features in the compressed representation

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Training the autoencoder
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=16, shuffle=True, validation_split=0.2)

# Get reconstruction error
reconstructions = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructions, 2), axis=1)

# Set a threshold for anomaly detection
threshold = np.percentile(mse, 95)

# Flag anomalies
log_df['Anomaly'] = mse > threshold

# Show anomalies
anomalies = log_df[log_df['Anomaly']]
print(anomalies)


Epoch 1/50
1512/1512 [==============================] - 5s 2ms/step - loss: 1.1538 - val_loss: 0.6241
Epoch 2/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.9911 - val_loss: 0.5451
Epoch 3/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.9420 - val_loss: 0.5108
Epoch 4/50
1512/1512 [==============================] - 4s 3ms/step - loss: 0.9236 - val_loss: 0.4959
Epoch 5/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.9004 - val_loss: 0.4701
Epoch 6/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.8278 - val_loss: 0.3903
Epoch 7/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.7927 - val_loss: 0.3785
Epoch 8/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.7845 - val_loss: 0.3717
Epoch 9/50
1512/1512 [==============================] - 4s 2ms/step - loss: 0.7791 - val_loss: 0.3670
Epoch 10/50
1512/1512 [==============================] - 3s 2ms/step - loss: 0.775